In [13]:
import pandas as pd
import numpy as np

In [14]:
import librosa

----
## MFCC on noise only

In [30]:
max_pad_len = 174

def extract_features(file_name):
   
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        pad_width = max_pad_len - mfccs.shape[1]
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
        
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None 
     
    return mfccs

In [31]:
# Set the path to the full UrbanSound dataset 
fulldatasetpath = 'UrbanSound8K/audio/'

metadata = pd.read_csv('UrbanSound8K/metadata/UrbanSound8K.csv')

features = []

# Iterate through each sound file and extract the features 
counter = 0
for index, row in metadata.iterrows():    
    file_name = fulldatasetpath +'fold'+str(row["fold"])+'/'+str(row["slice_file_name"])
    
    class_label = row["class"]
    data = extract_features(file_name)
    
    features.append([data, class_label])
    if counter % 100 == 0:
        print(counter)
    counter += 1

# Convert into a Panda dataframe 
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])

print('Finished feature extraction from ', len(featuresdf), ' files')

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
Finished feature extraction from  8732  files


In [73]:
featuresdf.head(3)

,feature,class_label
0,"[[-306.77255, -177.59209, -99.13616, -65.97198...",dog_bark
1,"[[-457.69534, -451.0248, -450.68613, -445.0000...",children_playing
2,"[[-468.0367, -467.42264, -481.04654, -486.5948...",children_playing


In [134]:
featuresdf['feature'][0].shape

(40, 174)

In [136]:
test_featuresdf = featuresdf.copy()

In [137]:
# each value in feature is a (40,174) numpy array, in order to pass the data to Google Colab, it has to be saved as a .csv file. However, pd.to_csv ruins
# the numpy format. What I did here is faltten the 2D numpy array to a list and convert the list to string with only number and comma
# later in the Google Colab, read the data into dataframe and using numpy.fromstring convert it back to numpy array and reshape it back to (40,174), since
# every original value has a fixed lenth, it won't cause any problem.

# Disadvantage: the file is really large (1.12GB) compare the .to_csv directly (5MB+).

test_featuresdf['new_feature'] = test_featuresdf.apply(lambda x: str(x['feature'].flatten().tolist()).strip('[]'),axis=1)

In [138]:
test_featuresdf = test_featuresdf[['new_feature','class_label']]

In [139]:
test_featuresdf.head(3)

,new_feature,class_label
0,"-306.7725524902344, -177.5920867919922, -99.13...",dog_bark
1,"-457.6953430175781, -451.0248107910156, -450.6...",children_playing
2,"-468.0367126464844, -467.4226379394531, -481.0...",children_playing


In [127]:
test_featuresdf.to_csv('mfcc_noise_only.csv',index=False)

---
# Combine my voice with noise and get the mfccs

In [150]:
def combine_voice_array(noise,target_voice):
    len_a = len(noise)
    len_b = len(target_voice)
    a = noise.copy()
    b = target_voice.copy()
    if len_a < len_b:
        b[:len_a] += a
        return (b / 2)
    else:
        a[:len_b] += b
        return (a/ 2)

In [198]:
max_pad_len = 222

def extract_combined_features(file_name):
    
    try:
        noise, noise_rate = librosa.load(file_name, res_type='kaiser_fast') 
        lower_coef = noise.max() / 0.07
        noise = noise/lower_coef
        
        combined_voice = combine_voice_array(noise,my_voice)
        
        mfccs = librosa.feature.mfcc(y=combined_voice, sr=my_voice_rate, n_mfcc=40)
        pad_width = max_pad_len - mfccs.shape[1]
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
        
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None 
     
    return mfccs

In [199]:
# Set the path to the full UrbanSound dataset 
fulldatasetpath = 'UrbanSound8K/audio/'

metadata = pd.read_csv('UrbanSound8K/metadata/UrbanSound8K.csv')

combined_voice_features = []

my_voice,my_voice_rate = librosa.load('my_voice_sample.wav', res_type='kaiser_fast')

# Iterate through each sound file and extract the features 
counter = 0
for index, row in metadata.iterrows():    
    file_name = fulldatasetpath +'fold'+str(row["fold"])+'/'+str(row["slice_file_name"])
    
    class_label = row["class"]
    data = extract_combined_features(file_name)
    
    combined_voice_features.append([data, class_label])
    
    
    if counter % 1000 == 0:
        print(counter)
    counter += 1

# Convert into a Panda dataframe 
combined_voice_features_df = pd.DataFrame(combined_voice_features, columns=['feature','class_label'])

print('Finished feature extraction from ', len(featuresdf), ' files')

0
1000
2000
3000
4000
5000
6000
7000
8000
Finished feature extraction from  8732  files


In [200]:
combined_voice_features_df.head(3)

,feature,class_label
0,"[[-615.8928, -501.51147, -418.0187, -379.61188...",dog_bark
1,"[[-431.9947, -425.3231, -423.77222, -418.19803...",children_playing
2,"[[-383.8137, -383.19577, -395.84634, -401.7457...",children_playing


In [203]:
combined_voice_features_df_temp = combined_voice_features_df.copy()

In [204]:
combined_voice_features_df_temp['new_feature'] = combined_voice_features_df_temp.apply(lambda x: str(x['feature'].flatten().tolist()).strip('[]'),axis=1)

In [205]:
combined_voice_features_df_temp = combined_voice_features_df_temp[['new_feature','class_label']]

In [206]:
combined_voice_features_df_temp.head(3)

,new_feature,class_label
0,"-615.892822265625, -501.511474609375, -418.018...",dog_bark
1,"-431.99468994140625, -425.3230895996094, -423....",children_playing
2,"-383.8136901855469, -383.1957702636719, -395.8...",children_playing


In [207]:
combined_voice_features_df_temp.to_csv('mfcc_combined_voice.csv',index=False)

## ZERO CROSSING rate

In [209]:
my_voice,my_voice_rate = librosa.load('my_voice_sample.wav', res_type='kaiser_fast')

In [215]:
mel_test = librosa.feature.melspectrogram(y=my_voice, sr=my_voice_rate)

In [213]:
mel_test.shape

(128, 222)

In [218]:
max_pad_len = 222

def mel_extract_combined_features(file_name):
    
    try:
        noise, noise_rate = librosa.load(file_name, res_type='kaiser_fast') 
        lower_coef = noise.max() / 0.07
        noise = noise/lower_coef
        
        combined_voice = combine_voice_array(noise,my_voice)
        
        mel = librosa.feature.melspectrogram(y=combined_voice, sr=my_voice_rate)
        pad_width = max_pad_len - mel.shape[1]
        mel = np.pad(mel, pad_width=((0, 0), (0, pad_width)), mode='constant')
        
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None 
     
    return mel

In [223]:
# Set the path to the full UrbanSound dataset 
fulldatasetpath = 'UrbanSound8K/audio/'

metadata = pd.read_csv('UrbanSound8K/metadata/UrbanSound8K.csv')

combined_voice_features_mel = []

my_voice,my_voice_rate = librosa.load('my_voice_sample.wav', res_type='kaiser_fast')

# Iterate through each sound file and extract the features 
counter = 0
for index, row in metadata.iterrows():    
    file_name = fulldatasetpath +'fold'+str(row["fold"])+'/'+str(row["slice_file_name"])
    
    class_label = row["class"]
    data = mel_extract_combined_features(file_name)
    
    combined_voice_features_mel.append([data, class_label])
    
    
    if counter % 1000 == 0:
        print(counter)
    counter += 1

# Convert into a Panda dataframe 
combined_voice_features_mel_df = pd.DataFrame(combined_voice_features_mel, columns=['feature','class_label'])

print('Finished feature extraction from ', len(featuresdf), ' files')

0
1000
2000
3000
4000
5000
6000
7000
8000
Finished feature extraction from  8732  files


In [224]:
combined_voice_features_mel_df.head(3)

,feature,class_label
0,"[[6.6912676e-08, 1.4397854e-07, 4.937666e-07, ...",dog_bark
1,"[[0.03327825, 0.06925758, 0.031367134, 0.01829...",children_playing
2,"[[0.01548944, 0.08860275, 0.3138474, 0.2061931...",children_playing


In [226]:
combined_voice_features_mel_df_temp = combined_voice_features_mel_df.copy()

In [227]:
combined_voice_features_mel_df_temp['new_feature'] = combined_voice_features_mel_df_temp.apply(lambda x: str(x['feature'].flatten().tolist()).strip('[]'),axis=1)

In [228]:
combined_voice_features_mel_df_temp = combined_voice_features_mel_df_temp[['new_feature','class_label']]

In [229]:
combined_voice_features_mel_df_temp.head(3)

,new_feature,class_label
0,"6.691267628866626e-08, 1.4397853931313875e-07,...",dog_bark
1,"0.03327824920415878, 0.06925757974386215, 0.03...",children_playing
2,"0.01548944041132927, 0.08860275149345398, 0.31...",children_playing


In [230]:
combined_voice_features_mel_df_temp.to_csv('mel_combined_voice.csv',index=False)

In [232]:
combined_voice_features_mel_df['feature'][0].shape

(128, 222)